In [0]:
# Widgets
dbutils.widgets.text("sub", "", "Subestação")
dbutils.widgets.text("feeder", "", "Alimentador")
dbutils.widgets.text("uni_tr_mt", "", "Rede BT")
sub = dbutils.widgets.get("sub")
feeder = dbutils.widgets.get("feeder")
uni_tr_mt = dbutils.widgets.get("uni_tr_mt")
random_state = 36

In [0]:
import os
import sys
import json
import numpy as np
import pandas as pd
from collections import defaultdict
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)

In [0]:
from Utils.MDMS import MDMS, config
mdms = MDMS(sub=sub, feeder=feeder, uni_tr_mt=uni_tr_mt, random_state=random_state)

In [0]:
data = {x.Name: x.Bus1.split(".")[0].split("_")[-1] for x in mdms.altdss.Load if x.Class%10 == 1}
seen = {}
result = {}
for key, value in data.items():
    if value not in seen:
        seen[value] = key
        result[key] = value
if not len(result) > 50:
    sys.exit("Execução interrompida")

In [0]:
cases = []
size = 3
for j in range(100): # ~ 100 scenarios
    customers = np.random.choice(list(result.keys()), size, replace=False)
    kw = np.round(np.random.uniform(low=1.50, high=3.00, size=size), 2)
    pf = np.round(np.random.uniform(low=0.85, high=1.00, size=size), 2)
    cases.append({
        "name": customers,
        "kw_by_phase": kw,
        "pf": pf,
        "shape_type": ["fixed"]*size,
        "working_period": [(int(np.random.choice(np.arange(9, 15))), int(np.random.choice(np.arange(17, 22)))) for x in range(size)],
        "theft_on_training": [False]*size,
    })


In [0]:
# Medições
meas = mdms.acquire() 

# MLP
from Models.MultilayerPerceptron import MLP, Training
engine = Training(
    model=MLP(
        input_size=mdms.inputs.shape[-1],
        hidden_layers=list(np.asarray(config.mlp.hidden_layers)*mdms.inputs.shape[-1]),
        output_size=mdms.outputs.shape[-1],
        activation_function=config.mlp.activation_function
    ), 
    inputs = mdms.training.get("inputs"), 
    outputs = mdms.training.get("outputs"), 
    random_state=random_state
)
mlp = engine.train(
    log_params={
        "sub": sub,
        "feeder": feeder,
        "uni_tr_mt": uni_tr_mt,
        "random_state": random_state
    },
    use_mlflow = False
)

# LR
from sklearn.linear_model import LinearRegression
from Models.LinearRegression import Training
model = LinearRegression()
inputs, outputs = mdms.training.get("inputs"), mdms.training.get("outputs") 
engine = Training(
    model=model, 
    inputs=inputs, 
    outputs=outputs,
    test_size=0.3,
    random_state=random_state
)
linear_reg = engine.train()
mse, r2 = engine.evaluate_model()
print(f'MSE: {mse:.6f}, R²: {r2:.6f}')


In [0]:
from Models.MultilayerPerceptron import NTLDetection
from Models.LinearRegression import Detector
output = defaultdict(list)
for i, case_ in enumerate(cases):
    
    # Restart mdms engine
    mdms = MDMS(sub=sub, feeder=feeder, uni_tr_mt=uni_tr_mt, random_state=random_state)

    # NTL
    meas = mdms.acquire(customers=case_) 

    # Detecção MLP
    MLPdeltaW = NTLDetection(
        model = mlp,
        inputs = mdms.inference.get("inputs"),
        outputs = mdms.inference.get("outputs"),
    ).execute(
        rn_threshold=6,
        r2_threshold=0.90,
    )

    # LR
    inputs, outputs = mdms.inference.get("inputs"), mdms.inference.get("outputs") 
    LRdeltaW = Detector(model=linear_reg, inputs=inputs, outputs=outputs).execute()

    # Params
    output = {
        "sub": sub,
        "feeder": feeder,
        "uni_tr_mt": uni_tr_mt,
        "random_state": random_state,
        "irregular_bus": json.dumps(mdms.irregs.bus_id.tolist()),
        "irregular_load": json.dumps(mdms.irregs.name.tolist()),
        "phases": json.dumps(dict(zip(mdms.irregs.bus_id.tolist(), mdms.irregs.num_phases.tolist()))),
        "bypassed_kw": json.dumps(dict(zip(mdms.irregs.bus_id.tolist(), mdms.irregs.kw_by_phase.tolist()))),
        "bypassed_pf": json.dumps(dict(zip(mdms.irregs.bus_id.tolist(), mdms.irregs.pf.tolist()))),
        "kwh_measured": json.dumps(dict(zip(mdms.irregs.bus_id.tolist(), mdms.irregs.kwh_measured.tolist()))),
        "kwh_theft": json.dumps(dict(zip(mdms.irregs.bus_id.tolist(), mdms.irregs.kwh_theft.tolist()))),
        "mlp_suspects": json.dumps(dict(zip(MLPdeltaW.bus_id.tolist(), MLPdeltaW.deltaW.tolist()))),
        "lr_suspects": json.dumps(dict(zip(LRdeltaW.bus_id.tolist(), LRdeltaW.deltaW.tolist())))
    }
    data = pd.DataFrame(output, index=[0])
    data = spark.createDataFrame(data)
    data.write.format("delta").mode("append").option("overwriteSchema", "true").saveAsTable("le41.bronze.phd_monte_carlo_multiple_ntl")



In [0]:
data.display()

In [0]:
data.display()